# llama2模型训练——数据处理，模型构建，模型训练及推理

## 参考 ：https://github.com/meta-llama/llama3/tree/main
## 数据集：https://modelscope.cn/datasets/AI-ModelScope/chinese-c4
## paper : 
### llama2: https://arxiv.org/abs/2307.09288
###         llama 3: https://arxiv.org/abs/2407.21783
## 本代码git: https://github.com/zhangjianzoujianghu18/learning-gpt.git

# 1 训练数据集加工 

In [22]:
import zstandard as zstd
import json
import io
import os
import glob
import os
import torch
from tqdm import tqdm
import tiktoken
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset, IterableDataset
from typing import Any, Optional, Tuple

# 定义一个生成器函数来读取 .jsonl.zst 文件
def read_jsonl_zst(file_path):
    with open(file_path, 'rb') as fh:
        dctx = zstd.ZstdDecompressor()
        stream_reader = dctx.stream_reader(fh)
        text_stream = io.TextIOWrapper(stream_reader, encoding='utf-8')
        for line in text_stream:
            yield json.loads(line)

# 定义一个生成器函数来读取所有 .jsonl.zst 文件
def read_all_jsonl_zst(files):
    for file_path in files:
        yield from read_jsonl_zst(file_path)


def process(example):
    ids = enc.encode_ordinary(example['text'])
    ids.append(enc.eot_token)
    return {'ids': ids, 'len': len(ids)}

class StreamingParquetDataset(torch.utils.data.IterableDataset):
    def __init__(self, jsonl_zst_files, split, block_size, num_proc=14):
        self.data_files = jsonl_zst_files
        self.split = split
        self.block_size = block_size
        self.num_proc = num_proc
        self.dataset = IterableDataset.from_generator( lambda: read_all_jsonl_zst(jsonl_zst_files))
#              load_dataset("arrow", data_files={split: data_files}, streaming=True)
        self.tokenized = self.dataset.map(process)
    def __iter__(self):
        for example in self.tokenized:
            ids = example['ids']
            for i in range(0, len(ids) - self.block_size, self.block_size):
                x = torch.tensor(ids[i:i + self.block_size], dtype=torch.int64)
                y = torch.tensor(ids[i + 1:i + 1 + self.block_size], dtype=torch.int64)
                yield x, y

# 示例函数：获取一个批次的数据
def get_batch(loader, device,device_type):
    for x, y in loader:
        if device_type == 'cuda':
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        else:
            x, y = x.to(device), y.to(device)
        yield x, y
        
def get_train_data_from_stream_data(data_path_root,enc,batch_size=32,block_size=128):
    block_size = block_size  # 根据你的模型设置合适的块大小
    batch_size = batch_size  # 根据你的硬件设置合适的批次大小
 
    # 查找所有 .jsonl.zst 文件
    jsonl_zst_files = glob.glob(f'{data_path_root}/*.jsonl.zst', recursive=True)

    # 创建数据集
    train_dataset = StreamingParquetDataset(jsonl_zst_files[:-1], 'train', block_size)
    val_dataset = StreamingParquetDataset([jsonl_zst_files[-1]], 'val', block_size)

    # 创建数据加载器
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    return train_loader,val_loader
enc = tiktoken.get_encoding("cl100k_base")


In [25]:

data_path_root = "/Users/wangaijun/pythoncode/github/data/text/chinese-c4"
batch_size=32
max_seq_len=512
# 查找所有 .jsonl.zst 文件

train_loader,val_loader=get_train_data_from_stream_data(data_path_root,enc,batch_size=batch_size,block_size=max_seq_len)


In [26]:
for x,y in train_loader:
    print("x shape:",x.shape," y shape: ",y.shape)
    print(x)
    print(y)
    break

x shape: torch.Size([32, 512])  y shape:  torch.Size([32, 512])
tensor([[16175,   121, 36651,  ..., 50266,   111, 50182],
        [15355,   100, 21589,  ..., 37767, 54493, 17599],
        [37795,   237, 84949,  ...,   238, 42506,  9174],
        ...,
        [14354,   606, 36969,  ...,   114, 71600, 76537],
        [12870,    97,   163,  ..., 43240, 19361, 41914],
        [43032,  1811, 56235,  ..., 13646, 17885,   245]])
tensor([[  121, 36651, 85315,  ...,   111, 50182, 15355],
        [  100, 21589,   242,  ..., 54493, 17599,   230],
        [  237, 84949,   222,  ..., 42506,  9174, 20713],
        ...,
        [  606, 36969,    64,  ..., 71600, 76537, 67178],
        [   97,   163,   123,  ..., 19361, 41914, 43032],
        [ 1811, 56235, 32943,  ..., 17885,   245, 43240]])


# 2 模型构建

<div style="text-align: center;">
  <img src="images/transformer.png" alt="Image" style="width:600px;">
</div>

In [29]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

# @dataclass
# class ModelArgs:
#     dim: int = 4096
#     n_layers: int = 32
#     n_heads: int = 32
#     n_kv_heads: Optional[int] = None
#     vocab_size: int = -1
#     multiple_of: int = 256  # make SwiGLU hidden layer size multiple of large power of 2
#     ffn_dim_multiplier: Optional[float] = None
#     norm_eps: float = 1e-5
#     rope_theta: float = 500000

#     max_batch_size: int = 32
#     max_seq_len: int = 2048


class config:
    block_size=256
    vocab_size=100277
    n_layers=3
    n_head=4
    n_embd=128
    dropout=0.0
    bias=False
    norm_eps = 1e-5
    multiple_of: int = 20
    dim=128
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)

train_loader,val_loader=get_train_data_from_stream_data(data_path_root,enc,batch_size=batch_size,block_size=config.block_size)
for x,y in train_loader:
    print("x shape:",x.shape," y shape: ",y.shape)
    print(x)
    print(y)
    break

x shape: torch.Size([32, 256])  y shape:  torch.Size([32, 256])
tensor([[16175,   121, 36651,  ...,   255, 25781,   252],
        [31867,  5486, 29504,  ..., 50266,   111, 50182],
        [15355,   100, 21589,  ...,  3922, 81258, 66201],
        ...,
        [ 8107,    22,  9953,  ..., 97150, 64803, 23602],
        [  232, 17792, 27384,  ..., 83799,  8239,   123],
        [ 3922,  6701,   249,  ...,  4468,    24,  8107]])
tensor([[  121, 36651, 85315,  ..., 25781,   252, 31867],
        [ 5486, 29504, 14309,  ...,   111, 50182, 15355],
        [  100, 21589,   242,  ..., 81258, 66201, 86436],
        ...,
        [   22,  9953,  3922,  ..., 64803, 23602,   232],
        [17792, 27384, 30537,  ...,  8239,   123,  3922],
        [ 6701,   249, 67178,  ...,    24,  8107,    16]])


## 2.1 embedding 

In [30]:
wte = nn.Embedding(config.vocab_size, config.n_embd)
# wpe = nn.Embedding(config.block_size, config.n_embd)// 改为旋转位置编码
drop = nn.Dropout(config.dropout)
ln_f = nn.LayerNorm(config.n_embd, bias=config.bias)

In [31]:
x_embd=wte(x)
print("x_embd shape: ",x_embd.shape)
x_embd_ln=ln_f(x_embd)
x_embd_ln.shape

x_embd shape:  torch.Size([32, 256, 128])


torch.Size([32, 256, 128])

## 2.2 FFN层

In [32]:
class FeedForward(nn.Module):
    def __init__(self, dim: int, hidden_dim: int, multiple_of: int, dropout: float):
        super().__init__()
        hidden_dim = int(2 * hidden_dim / 3)
        hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)
        self.w1 = nn.Linear(dim, hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, dim, bias=False)
        self.w3 = nn.Linear(dim, hidden_dim, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.dropout(self.w2(F.silu(self.w1(x)) * self.w3(x)))

mlp=FeedForward(config.n_embd,config.n_embd,config.multiple_of,config.dropout)(x_embd_ln)
mlp.shape

torch.Size([32, 256, 128])

In [33]:
class RMSNorm(torch.nn.Module):
    def __init__(self, dim: int, eps: float):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x):
        output = self._norm(x.float()).type_as(x)
        return output * self.weight
rms=RMSNorm(config.n_embd,config.norm_eps)

In [34]:
rms=rms(mlp)
rms.shape

torch.Size([32, 256, 128])

## 2.4 旋转位置编码

### B站视频： https://www.bilibili.com/video/BV14JB1Y3E6i/?spm_id_from=333.999.0.0&vd_source=6f858f592d89bed2a97f471f3232ad57

In [35]:
import torch
import math

def get_rotary_position_embedding(seq_len, dim):
    position = torch.arange(0, seq_len, dtype=torch.float32).unsqueeze(1)
    div_term = 1. / (10000 ** (torch.arange(0, dim, 2).float() / dim))
    pe = torch.zeros(seq_len, dim)
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe
rope=get_rotary_position_embedding(config.block_size, int(config.n_embd/config.n_head))
print(rope)

tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  5.3317e-01,  ...,  1.0000e+00,
          1.7783e-04,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  9.0213e-01,  ...,  1.0000e+00,
          3.5566e-04,  1.0000e+00],
        ...,
        [ 9.9482e-01, -1.0162e-01, -7.8375e-01,  ...,  9.9680e-01,
          4.4975e-02,  9.9899e-01],
        [ 4.5200e-01, -8.9202e-01, -9.9420e-01,  ...,  9.9678e-01,
          4.5153e-02,  9.9898e-01],
        [-5.0639e-01, -8.6230e-01, -8.9845e-01,  ...,  9.9675e-01,
          4.5331e-02,  9.9897e-01]])


In [36]:
def apply_rotary_position_embedding(x, rope):
    # 分离奇数和偶数索引
    x1 = x[..., ::2]  # 偶数索引
    x2 = x[..., 1::2]  # 奇数索引
    # 从rope中获取相应的cosine和sine部分
    cos_pos = rope[:, 0::2].repeat((x.size(0), x.size(1), 1, 1))  # 偶数索引
    sin_pos = rope[:, 1::2].repeat((x.size(0), x.size(1), 1, 1))  # 奇数索引
    # 应用旋转位置编码
    return torch.cat([x1 * cos_pos - x2 * sin_pos, x2 * cos_pos + x1 * sin_pos], dim=-1)

x=rms.reshape(32,config.n_head,config.block_size,-1)
apply_rotary_position_embedding(x, rope).shape

torch.Size([32, 4, 256, 32])

## 2.5 attention

In [37]:
class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x,rope):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs) 
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # 添加旋转位置编码
        q=apply_rotary_position_embedding(q, rope)
        k= apply_rotary_position_embedding(k, rope)
        
        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

In [38]:
atten=CausalSelfAttention(config)(rms,rope)
atten.shape

torch.Size([32, 256, 128])

## 2.6 block

In [39]:
class TransformerBlock(nn.Module):
    def __init__(self, layer_id: int, config):
        super().__init__()
        self.n_heads = config.n_head
        self.dim = config.n_embd
        self.head_dim = config.n_embd // config.n_head
        self.attention = CausalSelfAttention(config)
        self.feed_forward = FeedForward(
            dim=config.n_embd,
            hidden_dim=1 * config.n_embd,
            multiple_of=config.multiple_of,
            dropout=config.dropout,
        )
        self.layer_id = layer_id
        self.attention_norm = RMSNorm(config.n_embd, eps=config.norm_eps)
        self.ffn_norm = RMSNorm(config.n_embd, eps=config.norm_eps)

    def forward(self, x,rope):
        h = x + self.attention(self.attention_norm(x), rope)
        out = h + self.feed_forward.forward(self.ffn_norm(h))
        return out


In [40]:
TransformerBlock(1,config)(rms,rope).shape

torch.Size([32, 256, 128])

In [41]:
class Transformer(nn.Module):
    last_loss: Optional[torch.Tensor]
    def __init__(self, params):
        super().__init__()
        self.params = params
        self.vocab_size = params.vocab_size
        self.n_layers = params.n_layers

        self.tok_embeddings = nn.Embedding(params.vocab_size, params.dim)
        self.dropout = nn.Dropout(params.dropout)
        self.layers = torch.nn.ModuleList()
        for layer_id in range(params.n_layers):
            self.layers.append(TransformerBlock(layer_id, params))
        self.norm = RMSNorm(params.dim, eps=params.norm_eps)
        self.output = nn.Linear(params.dim, params.vocab_size, bias=False)

        # share the unembedding parameters with the embedding parameters
        self.tok_embeddings.weight = self.output.weight # https://paperswithcode.com/method/weight-tying

        # some useful precompute for the RoPE relative positional embeddings
        # freqs_cos, freqs_sin = precompute_freqs_cis(self.params.dim // self.params.n_heads, self.params.max_seq_len)
        # self.register_buffer("freqs_cos", freqs_cos, persistent=False)
        # self.register_buffer("freqs_sin", freqs_sin, persistent=False)
        rope=get_rotary_position_embedding(config.block_size, int(config.n_embd/config.n_head))
        self.register_buffer("rope", rope, persistent=False)
        
        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('w3.weight') or pn.endswith('wo.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * params.n_layers))

        # Initialize attribute for the loss of the last forward call. This will be set if the forward is called with a targets tensor.
        self.last_loss = None

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def get_num_params(self):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        return n_params
        
    def forward(self, tokens: torch.Tensor, targets: Optional[torch.Tensor] = None) -> torch.Tensor:
        _bsz, seqlen = tokens.shape
        h = self.tok_embeddings(tokens)
        h = self.dropout(h)
        rope = self.rope[:seqlen]
    
        for layer in self.layers:
            h = layer(h, rope)
        h = self.norm(h)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.output(h)
            self.last_loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the output on the very last position
            logits = self.output(h[:, [-1], :]) # note: using list [-1] to preserve the time dim
            self.last_loss = None

        return logits,self.last_loss

In [42]:
llama2_model=Transformer(config)
print("模型参数：",llama2_model.get_num_params())
llama2_model

模型参数： 13148160


Transformer(
  (tok_embeddings): Embedding(100277, 128)
  (dropout): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-2): 3 x TransformerBlock(
      (attention): CausalSelfAttention(
        (c_attn): Linear(in_features=128, out_features=384, bias=False)
        (c_proj): Linear(in_features=128, out_features=128, bias=False)
        (attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
      (feed_forward): FeedForward(
        (w1): Linear(in_features=128, out_features=100, bias=False)
        (w2): Linear(in_features=100, out_features=128, bias=False)
        (w3): Linear(in_features=128, out_features=100, bias=False)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (attention_norm): RMSNorm()
      (ffn_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=128, out_features=100277, bias=False)
)

# 3 模型训练


In [43]:
# adamw optimizer
learning_rate = 6e-4 # max learning rate
max_iters = 600000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
optimizer = torch.optim.AdamW(llama2_model.parameters(), lr=learning_rate, betas=(beta1,beta2))


In [44]:
i=0
for x, y in get_batch(train_loader, device,"cpu"):
    # 在这里进行模型训练
    print(x.shape, y.shape)
    print(f"embedding weight step {i}",llama2_model.tok_embeddings.weight[:3,:5])
    logits, loss = llama2_model(x, y)
    loss.backward()
    optimizer.step()
    if i==3:
        break  # 只打印一个批次的数据
    i+=1

torch.Size([32, 256]) torch.Size([32, 256])
embedding weight step 0 tensor([[-1.3147e-02, -2.3653e-02, -2.3193e-02,  8.1439e-03, -3.8375e-03],
        [ 3.0824e-02, -2.1510e-02,  2.2791e-05, -2.2051e-02, -2.2887e-02],
        [ 3.2242e-03, -1.5278e-02,  2.0917e-02, -5.3500e-03,  3.0080e-04]],
       grad_fn=<SliceBackward0>)
torch.Size([32, 256]) torch.Size([32, 256])
embedding weight step 1 tensor([[-0.0125, -0.0231, -0.0226,  0.0087, -0.0044],
        [ 0.0314, -0.0209,  0.0006, -0.0215, -0.0235],
        [ 0.0038, -0.0147,  0.0215, -0.0048, -0.0003]],
       grad_fn=<SliceBackward0>)
torch.Size([32, 256]) torch.Size([32, 256])
embedding weight step 2 tensor([[-0.0120, -0.0225, -0.0220,  0.0093, -0.0050],
        [ 0.0320, -0.0204,  0.0012, -0.0209, -0.0241],
        [ 0.0044, -0.0141,  0.0221, -0.0042, -0.0009]],
       grad_fn=<SliceBackward0>)
torch.Size([32, 256]) torch.Size([32, 256])
embedding weight step 3 tensor([[-0.0114, -0.0220, -0.0215,  0.0099, -0.0056],
        [ 0.0317

torch.Size([32, 256])

# 4 模型推理

In [45]:
max_new_tokens=30
temperature=1.0
top_k=4
idx=torch.randint(0,10000,(2,4))
print("before idx:",idx)

with torch.no_grad():
    for _ in range(max_new_tokens):
        logits, _ = llama2_model(idx)
        logits = logits[:, -1, :] / temperature
        # optionally crop the logits to only the top k options
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float('Inf')
        # apply softmax to convert logits to (normalized) probabilities
        probs = F.softmax(logits, dim=-1)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1)
        # append sampled index to the running sequence and continue
        idx = torch.cat((idx, idx_next), dim=1)

idx

before idx: tensor([[9736, 7584, 3388, 2397],
        [2889, 7981, 6261, 8787]])


tensor([[ 9736,  7584,  3388,  2397, 27899, 97338, 97338, 57318, 58571, 65182,
         97338, 14415, 65182, 65182, 97338, 65182, 65182,  3045, 65182,  3045,
         12167, 97338, 97338, 65182, 65182, 65182, 39294, 39294, 65182,  3045,
         65182, 65182, 39294, 39294],
        [ 2889,  7981,  6261,  8787, 86636, 86636, 48389, 44959, 34522, 60240,
         60240, 34522, 34993, 34993, 17885, 34732, 90718, 90718, 35722, 75778,
         35722, 34732, 17885,  3922, 35722, 17885,  3922, 99046, 35722, 35722,
         35722,  3922,  3922,  4130]])

In [46]:
idx.shape

torch.Size([2, 34])